### Notebook to iterate through Logisitic Regression Model

In [136]:
# add justification for why Log Reg Model would be useful in this case

In [153]:
# import libraries
import pandas as pd
pd.options.display.max_columns = 50
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import sklearn
from sklearn.preprocessing import StandardScaler, binarize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
# add import for Decision Treees
from sklearn.feature_selection import RFECV, SelectKBest, f_regression
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, plot_confusion_matrix, roc_curve, auc, classification_report
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, GridSearchCV
import pickle

In [154]:
# loading in cleaned dataset
model_df = pd.read_csv('model_data.csv', index_col = 0)

In [139]:
model_df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,2,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,2,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,2,3,3,1,4,Returning_Visitor,True,False


In [140]:
# Creating Dummys Columns from Cat
model_df = pd.get_dummies(model_df, columns=['SpecialDay', 'Month', 'OperatingSystems','Browser','Region','TrafficType','VisitorType','Weekend'])

In [141]:
model_df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,Revenue,SpecialDay_0.0,SpecialDay_0.2,SpecialDay_0.4,SpecialDay_0.6,SpecialDay_0.8,SpecialDay_1.0,Month_2,Month_3,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,...,TrafficType_1,TrafficType_2,TrafficType_3,TrafficType_4,TrafficType_5,TrafficType_6,TrafficType_7,TrafficType_8,TrafficType_9,TrafficType_10,TrafficType_11,TrafficType_12,TrafficType_13,TrafficType_14,TrafficType_15,TrafficType_16,TrafficType_17,TrafficType_18,TrafficType_19,TrafficType_20,VisitorType_New_Visitor,VisitorType_Other,VisitorType_Returning_Visitor,Weekend_False,Weekend_True
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,False,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,False,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,False,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,False,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,False,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1


In [142]:
# Dummying 'Revenue'
model_df['Revenue'] = model_df['Revenue'].astype(int)

In [143]:
model_df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,Revenue,SpecialDay_0.0,SpecialDay_0.2,SpecialDay_0.4,SpecialDay_0.6,SpecialDay_0.8,SpecialDay_1.0,Month_2,Month_3,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,...,TrafficType_1,TrafficType_2,TrafficType_3,TrafficType_4,TrafficType_5,TrafficType_6,TrafficType_7,TrafficType_8,TrafficType_9,TrafficType_10,TrafficType_11,TrafficType_12,TrafficType_13,TrafficType_14,TrafficType_15,TrafficType_16,TrafficType_17,TrafficType_18,TrafficType_19,TrafficType_20,VisitorType_New_Visitor,VisitorType_Other,VisitorType_Returning_Visitor,Weekend_False,Weekend_True
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1


## Train-Test Split

In [144]:
# Split data to be used in the models
X = model_df.drop('Revenue', axis = 1)

# Create target variable
y = model_df['Revenue'] # y is the column we're trying to predict

In [145]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=11)

In [147]:
y_train.shape


(9247,)

In [148]:
sm = SMOTE(random_state=23)

X_train, y_train = sm.fit_sample(X_train, y_train)

In [149]:
y_train.shape

(15664,)

In [151]:
X_train.shape

(15664, 80)

## Baseline Logistic Regression Model

In [133]:
baseline_log_reg = LogisticRegression(solver='liblinear',max_iter=500, verbose=1, n_jobs=-1)

baseline_log_reg.fit(X_train, y_train)

log_reg_pred_train = baseline_log_reg.predict(X_train)
log_reg_pred_test = baseline_log_reg.predict(X_test)

print('Baseline Logistic Regression Model')

print("Training F1 Score: ", metrics.f1_score(y_train, log_reg_pred_train))
print("Testing F1 Score: ", metrics.f1_score(y_test, log_reg_pred_test))

print("Training Recall: ", metrics.recall_score(y_train, log_reg_pred_train))
print("Testing Recall: ", metrics.recall_score(y_test, log_reg_pred_test))

print("Training Accuracy Score:",metrics.accuracy_score(y_train, log_reg_pred_train))
print("Testing Accuracy Score:",metrics.accuracy_score(y_test, log_reg_pred_test))

print("Training ROC, AUC Score:",metrics.roc_auc_score(y_train, log_reg_pred_train))
print("Testing ROC, AUC Score:",metrics.roc_auc_score(y_test, log_reg_pred_test))

print("Training Precision Score:",metrics.precision_score(y_train, log_reg_pred_train))
print("Testing Precision Score:",metrics.precision_score(y_test, log_reg_pred_test))

/Users/jefelito/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1353: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  warnings.warn("'n_jobs' > 1 does not have any effect when"


[LibLinear]Baseline Logistic Regression Model
Training F1 Score:  0.5111111111111111
Testing F1 Score:  0.4743411927877948
Training Recall:  0.3901060070671378
Testing Recall:  0.34685598377281945
Training Accuracy Score: 0.8858008002595437
Testing Accuracy Score: 0.8770677911125527
Training ROC, AUC Score: 0.6827317573640082
Testing ROC, AUC Score: 0.6624241308825486
Training Precision Score: 0.7409395973154362
Testing Precision Score: 0.75


In [92]:
# adding scaler
scaler = StandardScaler()  
scaler.fit(X_train)

X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)

## Creating 'Real' Model

In [99]:
balanced_log = LogisticRegression(class_weight='balanced', solver='lbfgs', verbose=1, n_jobs=-1)

balanced_log.fit(X_train, y_train)

balanced_preds = balanced_log.predict(X_test)

print("Testing F1 Score:",metrics.f1_score(y_test, balanced_preds))
print("Testing Accuracy Score:",metrics.accuracy_score(y_test, balanced_preds))
print("Testing ROC,AUC Score:",metrics.roc_auc_score(y_test, balanced_preds))
print("Testing Precision Score:",metrics.precision_score(y_test, balanced_preds))
print("Testing Recall Score:",metrics.recall_score(y_test, balanced_preds))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Testing F1 Score: 0.9398443029016277
Testing Accuracy Score: 0.9724294518326305
Testing ROC,AUC Score: 0.9703223227455754
Testing Precision Score: 0.9146005509641874
Testing Recall Score: 0.9665211062590975


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.9s finished


In [134]:
cross_val_score(balanced_log, X, y, cv=10, scoring='f1_score').mean()

ValueError: 'f1_score' is not a valid scoring value. Use sorted(sklearn.metrics.SCORERS.keys()) to get valid options.

# Synthetic Minority Oversampling (SMOTE)

In [97]:
from imblearn.over_sampling import SMOTE

In [115]:
# smote = SMOTE()

# X_train_resampled, y_train_resampled = smote.fit_sample(X_train, y_train)

# smote_log = LogisticRegression(penalty='l1',solver='saga',verbose=1, n_jobs=-1)

# smote_log = smote_log.fit(X_train_resampled, y_train_resampled)


# y_test = smote_log.predict(X_test)

# smote_log_preds = smote_log.decision_function(X_test)

# fpr, tpr, thresholds = roc_curve(y_test, smote_log_preds)


# print("Testing ROC,AUC Score:",metrics.roc_auc_score(y_test, smote_log_preds))


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


max_iter reached after 2 seconds
Testing ROC,AUC Score: 1.0


/Users/jefelito/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    2.1s finished


In [ ]:
# sm = SMOTE(random_state=23)
# X_train, y_train = sm.fit_sample(X_train, y_train)

In [117]:
cross_val_score(smote_log, X, y, cv=10, scoring='roc_auc').mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1

0.34848747043071404

In [102]:
pd.DataFrame([fpr,tpr], index=['FPR','TPR'], columns = thresholds).T.sort_index()

,FPR,TPR
-13.604047,1.000000,1.000000
-10.682514,0.992070,1.000000
-10.674029,0.991235,1.000000
-5.307526,0.951169,1.000000
-5.307082,0.950334,1.000000
-5.108667,0.943239,1.000000
-5.100539,0.942404,1.000000
-5.091566,0.941987,1.000000
-5.079461,0.941152,1.000000
-5.018088,0.937396,1.000000


In [105]:
def evaluate_threshold(threshold):
    print('Sensitivity:', tpr[thresholds > threshold][-1])
    print('Specificity:', 1 - fpr[thresholds > threshold][-1])

In [106]:
evaluate_threshold(0.5)

Sensitivity: 0.0014534883720930232
Specificity: 1.0
